<a href="https://colab.research.google.com/github/notmanan/Depression-Detection-Through-Multi-Modal-Data/blob/master/CNN__MCA_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connecting Drive to Google Colab**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

**Loading, Data Preprocessing for video**

In [0]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

def processData(data):
    # print(data)
    X = data.iloc[:,:].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
    for i in range(len(X)):
        if(isinstance(X[i][5],str) or isinstance(X[i][7],str)):
            X[i] = np.zeros((1, X.shape[1]))
            print("se" , end = " ")
    return X

def getData(filename):
    data = pd.read_csv(filename,delimiter=',', engine='python')
    X = processData(data)
    return X

def makeDataPoint(patientNo):
    file1 = (patientNo)+"_CLNF_AUs.txt"
    file2 = (patientNo)+"_CLNF_features.txt"
    file3 = (patientNo)+"_CLNF_features3D.txt"
    file4 = (patientNo)+"_CLNF_gaze.txt"
    file5 = (patientNo)+"_CLNF_hog.txt"
    file6 = (patientNo)+"_CLNF_pose.txt"

    X1 = getData(file1)
    X2 = getData(file2)
    X3 = getData(file3)
    X4 = getData(file4)
    X6 = getData(file6)

    X = np.concatenate((X1, X2, X3, X4, X6), 1)
    return X

def scale_down(X):
  X_new = []
  size = 2
  for i in range(int(X.shape[0]/size)):
    cur_row = X[i*size]
    for j in range(1,size):
      if(i+j < X.shape[0]):
        cur_row += X[i+j]
    cur_row = cur_row/size
    X_new.append(cur_row)
  X_new = np.array(X_new)
  return X_new

# def decrease_size(X):
#   size = 10000
#   print("Size: " + str(X.shape[0]))
#   if(X.shape[0]<size):
#     dif = size - X.shape[0] 
#     temp = np.zeros((dif,X.shape[1]))
#     X = np.concatenate((X,temp),axis = 0)
#   if(X.shape[0]>size):
#     dif = X.shape[0] - size
#     while(dif>0):
#       index = np.random.randint(0,X.shape[0]-1)
#       for i in range(X.shape[1]):
#         X[index][i] = (X[index][i] + X[index+1][i])/2
#       X = np.delete(X,index+1,axis = 0)
#       dif = X.shape[0] - size
#   return X

def decrease_size(X):
  size = 10000
  # print("Size: " + str(X.shape[0]))
  if(X.shape[0] < size):
    dif = size - X.shape[0] 
    temp = np.zeros((dif,X.shape[1]))
    X = np.concatenate((X,temp),axis = 0)
  elif(X.shape[0] > size):
    X = X[:10000, :]
    # print("New Shape" + str(X.shape[0]))
  return X

def makeDataset(location, folder):
    file  = np.array(pd.read_csv(location,delimiter=',',encoding='utf-8'))[:, 0:2]
    X_temp = []
    Y_temp = []
    for i in range(len(file)):
      patientID = (str(int(file[i][0])))
      print(patientID)
      string = '/content/drive/My Drive/MCA Dataset/' + folder +"/"+ patientID
      XT = makeDataPoint(string)
      XT = scale_down(XT)
      XT = decrease_size(XT)
      X_temp.append(XT)
      Y_temp.append(int(file[i][1]))
    Y_temp = np.asarray(Y_temp)
    return X_temp, Y_temp

print("train")
X_train, Y_train = makeDataset('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv', 'train_data')

print("dev")
X_dev, Y_dev = makeDataset('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv', 'dev_data')

print("test")
X_test, Y_test = makeDataset('/content/drive/My Drive/MCA Dataset/full_test_split.csv', 'test_data')



**CNN on dataset**

In [0]:
YtrainTotal = np.concatenate((Y_train, Y_dev))
XtrainTotal = X_train + X_dev 
XtrainTotal = np.asarray(XtrainTotal, dtype=np.float32)

from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv1D(300, 3, input_shape = (10000, 388), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling1D(pool_size = 2))

classifier.add(Conv1D(150, 3, activation = 'relu'))
classifier.add(MaxPooling1D(pool_size = 2))

# Adding a second convolutional layer
classifier.add(Conv1D(75, 3, activation = 'relu'))
classifier.add(MaxPooling1D(pool_size = 2))

classifier.add(Conv1D(32, 3, activation = 'relu'))
classifier.add(MaxPooling1D(pool_size = 2))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

from keras import metrics

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy')
class_weight = { 0: 0.3, 1:0.7}
classifier.fit(XtrainTotal, YtrainTotal, epochs=5)


In [7]:
# score = classifier.evaluate(X_test, Y_test)
from sklearn.metrics import classification_report
# print(classification_report(Y_test,classifier.predict(np.asarray(X_test))))
print("On Test Set: ")
print(classification_report(Y_test,classifier.predict(np.asarray(X_test))))

On Test Set: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        29
           1       0.29      1.00      0.45        12

    accuracy                           0.29        41
   macro avg       0.15      0.50      0.23        41
weighted avg       0.09      0.29      0.13        41



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
print(classifier.predict(np.asarray(X_test)))

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
